## а) Создать класс «Сотрудник» с полями «Имя», «Фамилия», «Должность» и «Зарплата». Создать хеш-таблицу для хранения объектов класса «Сотрудник» по ключу — табельному номеру.
## б) Написать функцию для удаления всех дубликатов из хеш-таблицы.

In [5]:
# Хеш таблица - это просто словарь, но в другом виде, вместо ключей - индекс. Если индекс больше чем длина хеш-таблицы - 
# то берется остаток от деления индекса на длину
class Employee:
    # Конструктор класса Employee инициализирует поля имени, фамилии, должности и зарплаты
    def __init__(self, first_name, last_name, position, zarplata):
        self.first_name = first_name
        self.last_name = last_name
        self.position = position
        self.zarplata = zarplata

    # Метод для представления объекта Employee в виде строки
    def __repr__(self):
        return f"Employee({self.first_name}, {self.last_name}, {self.position}, {self.zarplata})"

class EmployeeHashTable:
    # Конструктор класса EmployeeHashTable инициализирует размер таблицы и саму таблицу как список заданного размера
    def __init__(self, size=5):
        self.size = size
        self.table = [None] * size

    # Метод для вычисления индекса по табельному номеру
    def hash(self, employee_id):
        return employee_id % self.size

    # Метод для добавления сотрудника в хеш-таблицу по табельному номеру
    def add_employee(self, employee_id, employee):
        index = self.hash(employee_id)  # Вычисляем индекс в таблице
        if self.table[index] is None:
            self.table[index] = []  # Если по индексу пусто, создаем новый список
        self.table[index].append((employee_id, employee))  # Добавляем сотрудника в список по индексу

    # Метод для получения сотрудника из хеш-таблицы по табельному номеру
    def get_employee(self, employee_id):
        index = self.hash(employee_id)  # Вычисляем индекс в таблице
        if self.table[index] is not None:
            for eid, emp in self.table[index]:  # Ищем сотрудника в списке по индексу
                if eid == employee_id:
                    return emp  # Возвращаем сотрудника, если найден
        return None  # Возвращаем None, если сотрудник не найден

    # Метод для удаления всех дубликатов из хеш-таблицы
    def delete_duplicates(self):
        unique_entries = {}  # Создаем словарь для уникальных записей
        for index in range(self.size):
            if self.table[index] is not None:
                for employee_id, employee in self.table[index]:
                    if employee_id not in unique_entries:
                        unique_entries[employee_id] = employee  # Добавляем уникальную запись в словарь
        self.table = [None] * self.size  # Очищаем таблицу
        for employee_id, employee in unique_entries.items():
            self.add_employee(employee_id, employee)  # Добавляем уникальные записи обратно в таблицу

# Пример использования:
employee_table = EmployeeHashTable()
employee1 = Employee("John", "Doe", "Developer", 50000)
employee2 = Employee("Jane", "Smith", "Manager", 60000)
employee_table.add_employee(0, employee1)  # Добавляем сотрудника с табельным номером 0
employee_table.add_employee(1, employee2)  # Добавляем сотрудника с табельным номером 1
employee_table.add_employee(0, employee1)  # Добавляем дубликат сотрудника с табельным номером 0

print("До удаления дубликатов:")
print(employee_table.table)  # Выводим таблицу до удаления дубликатов
employee_table.delete_duplicates()  # Удаляем дубликаты
print("После удаления дубликатов:")
print(employee_table.table)  # Выводим таблицу после удаления дубликатов

До удаления дубликатов:
[[(0, Employee(John, Doe, Developer, 50000)), (0, Employee(John, Doe, Developer, 50000))], [(1, Employee(Jane, Smith, Manager, 60000))], None, None, None]
После удаления дубликатов:
[[(0, Employee(John, Doe, Developer, 50000))], [(1, Employee(Jane, Smith, Manager, 60000))], None, None, None]


## в) Реализуйте хеш-таблицу для хранения информации о книгах в библиотеке. Ключом является ISBN книги, значение — объект, содержащий информацию о книге (название, автор, количество экземпляров и т.д.). Используйте метод разрешения коллизий методом открытой адресации с квадратичным пробированием.

In [7]:
# метод разрешения коллизий - метод, который помогает поменять индекс элемента, если он уже был использован
# метод открытой адресации с квадратичным пробированием - метод измениния индекса через квадратичную функцию, который 
# используется столько раз, 
# пока не найдется пустой индекс (квадратичная функция: новый_индекс = индекс_(i-1)_попытки + i^2, где i - номер попытки)

class Book:
    # Конструктор класса Book инициализирует поля названия, автора и количества экземпляров
    def __init__(self, nazvanie, author, copies):
        self.nazvanie = nazvanie
        self.author = author
        self.copies = copies

    # Метод для удобного представления объекта Book в виде строки
    def __repr__(self):
        return f"Book({self.nazvanie}, {self.author}, {self.copies})"

class BookHashTable:
    # Конструктор класса BookHashTable инициализирует размер таблицы и саму таблицу как список заданного размера
    def __init__(self, size=5):
        self.size = size
        self.table = [None] * size

    #  метод для вычисления хеша по ISBN
    def hash(self, isbn):
        return isbn % self.size

    # метод для вычисления нового индекса при коллизии методом квадратичного пробирования
    def probe(self, index, step):
        return (index + step ** 2) % self.size

    # Метод для добавления книги в хеш-таблицу по ISBN
    def add_book(self, isbn, book):
        index = self.hash(isbn)  # Вычисляем начальный индекс
        step = 1
        while self.table[index] is not None and self.table[index][0] != isbn:
            index = self.probe(index, step)  # Вычисляем новый индекс при коллизии
            step += 1
        self.table[index] = (isbn, book)  # Добавляем книгу в таблицу по вычисленному индексу

    # Метод для получения книги из хеш-таблицы по ISBN
    def get_book(self, isbn):
        index = self.hash(isbn)  # Вычисляем начальный индекс
        step = 1
        while self.table[index] is not None:
            if self.table[index][0] == isbn:
                return self.table[index][1]  # Возвращаем книгу, если ISBN совпадает
            index = self.probe(index, step)  # Вычисляем новый индекс при коллизии
            step += 1
        return None  # Возвращаем None, если книга не найдена

# Пример использования:
book_table = BookHashTable()
book1 = Book("1984", "Оруэл", 5)
book2 = Book("Евгений Онегин", "Пушкин", 3)
book_table.add_book(123456789, book1)  # Добавляем книгу с ISBN 123456789
book_table.add_book(987654321, book2)  # Добавляем книгу с ISBN 987654321

print(book_table.get_book(123456789))  # Получаем книгу по ISBN 123456789
print(book_table.get_book(987654321))  # Получаем книгу по ISBN 987654321

Book(1984, Оруэл, 5)
Book(Евгений Онегин, Пушкин, 3)
